## Ensemble of Classifiers -  Draft 1 ##

#### An array of classifiers are trained & evaluated concurrently on different subsets of the input data. ####

Gestures files are separated by each axis of each joint (e.g. left_hand trains 3 separate classifiers- one for x-axis, one for y-axis and one for z-axis).

Crossvalidation split is performed separately on the gesture data for each joint. This means that left_hand_x is training on a different training set than left_hand_y or left_hand_z. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D
from sklearn import naive_bayes as nb
from sklearn.cross_validation import train_test_split

In [2]:
raw = pd.read_csv('MicrosoftGestureDataset-RC/data/P1_1_12A_p21.csv',sep=' ',header=None)
raw=raw.drop(xrange(0,20),axis=0).drop(0,axis=1).drop((xrange(4,80,4)),axis=1).drop(80,axis=1)

In [3]:
column_names=['hip_center_x','hip_center_y','hip_center_z',
              'spine_x','spine_y','spine_z','shoulder_center_x',
              'shoulder_center_y','shoulder_center_z','head_x',
              'head_y','head_z','shoulder_left_x','shoulder_left_y',
             'shoulder_left_z','elbow_left_x','elbow_left_y','elbow_left_z',
             'wrist_left_x','wrist_left_y','wrist_left_z','hand_left_x',
              'hand_left_y','wrist_left_z','shoulder_right_x','shoulder_right_y',
             'shoulder_right_z','elbow_right_x','elbow_right_y','elbow_right_z',
             'wrist_right_x','wrist_right_y','wrist_right_z','hand_right_x',
             'hand_right_y','hand_right_z','hip_left_x','hip_left_y','hip_left_z',
             'knee_left_x','knee_left_y','knee_left_z','ankle_left_x','ankle_left_y',
             'ankle_left_z','foot_left_x','foot_left_y','foot_left_z','hip_right_x',
             'hip_right_y','hip_right_z','knee_right_x','knee_right_y','knee_right_z',
             'ankle_right_x','ankle_right_y','ankle_right_z','foot_right_x','foot_right_y',
             'foot_right_z']

raw.columns=column_names


60

In [4]:
# create DataFrame for each axis of each joint
joints = {k: pd.DataFrame([]) for k in raw.columns }


In [5]:
gestures = pd.DataFrame()
labels = pd.Series()
labels_array=[]
sizes = [1];
i=0
for root, dirs, files in os.walk("MicrosoftGestureDataset-RC/data"):
        for f in files:
            if f.endswith(".csv"):
                temp = pd.read_csv(os.path.join(root,f),sep=' ',header=None).drop(xrange(0,20),axis=0).drop(0,axis=1).drop((xrange(4,80,4)),axis=1).drop(80,axis=1)
                temp
                temp.columns=column_names
                for joint_name in temp.columns:
                    current_timeseries = temp[joint_name]
                    current_timeseries.name = i
                    joints[joint_name] = joints[joint_name].append(current_timeseries)
                labels_array.append([f[f.find('_',3)+1:f.rfind('_')].replace('A',''),i])
                sizes.append(len(temp))
#                 print f[f.find('_',3)+1:f.rfind('_')].replace('A','');
                i=i+1;
#                 if i>100:
#                     break;
print str(i)+" gesture files loaded"



594 gesture files loaded


""


In [61]:
# gestureLabel is the correct label for the recorded gesture
# recordNumber is an ID for matching up records to their correct label after crossvalidation split
labels_df

,gestureLabel,recordNumber
0,5,0
1,8,1
2,1,2
3,7,3
4,6,4
5,7,5
6,12,6
7,7,7
8,10,8
9,9,9


In [64]:
(joints['wrist_left_x']).shape

(594, 519)

In [65]:
(joints['wrist_left_y']).shape

(594, 519)

In [6]:
labels_df = pd.DataFrame(labels_array,columns=['gestureLabel','recordNumber'])
classifiers = {k: nb.GaussianNB() for k in raw.columns }

# clean records to ~hopefully~ fix ValueError on gnb.predict()
for j in joints:
    joints[j] = joints[j].dropna(axis=1,how='any').reset_index().T.drop('index').T

In [7]:
# training/test set split
training_sets = {}
test_sets = {}
for c in classifiers:
    train, test = train_test_split(joints[c], test_size = 0.2);
    training_sets[c]=train;
    test_sets[c]=test;
    

In [8]:
training_sets['ankle_left_x'].shape

(475, 519)

In [67]:
test_sets['ankle_left_x'].shape

(119, 519)

In [87]:
# Fit separate classifiers for each joint
i = 0
for c in classifiers:
    try:
        classifiers[c] = classifiers[c].fit(training_sets[c],labels_df['gestureLabel'][:len(training_sets[c])])
        print classifiers[c].class_prior_
        print 'classifier #'+str(i)+' trained'
        
    except ValueError as e:
        print(e)
    i=i+1

[ 0.09263158  0.08        0.08210526  0.08421053  0.08631579  0.07157895
  0.07578947  0.07157895  0.08842105  0.09473684  0.08631579  0.08631579]
classifier #0 trained
[ 0.09263158  0.08        0.08210526  0.08421053  0.08631579  0.07157895
  0.07578947  0.07157895  0.08842105  0.09473684  0.08631579  0.08631579]
classifier #1 trained
[ 0.09263158  0.08        0.08210526  0.08421053  0.08631579  0.07157895
  0.07578947  0.07157895  0.08842105  0.09473684  0.08631579  0.08631579]
classifier #2 trained
[ 0.09263158  0.08        0.08210526  0.08421053  0.08631579  0.07157895
  0.07578947  0.07157895  0.08842105  0.09473684  0.08631579  0.08631579]
classifier #3 trained
[ 0.09263158  0.08        0.08210526  0.08421053  0.08631579  0.07157895
  0.07578947  0.07157895  0.08842105  0.09473684  0.08631579  0.08631579]
classifier #4 trained
Found arrays with inconsistent numbers of samples: [    594 1155433]
[ 0.09263158  0.08        0.08210526  0.08421053  0.08631579  0.07157895
  0.07578947 

In [89]:
results = {}
for c in classifiers:
# Wrist_Left_Z not parsing correctly
    if c!='wrist_left_z':
        results[c] = classifiers[c].predict(test_sets[c]);


In [100]:
# results checking per joint per axis
for r in results:
    true_labels = []
    z=0;
    for t in test_sets[r].index:
        true_labels.append(int(labels_df[labels_df['recordNumber']==t]['gestureLabel']))
        z=z+1
    output = results[r].astype(int)
    true_labels = np.array(true_labels)
    print r+' '+str(np.where(true_labels==output))
    print '\r\n'

hand_right_y (array([ 16,  20,  40,  42,  50,  53,  74,  84,  99, 115]),)


hand_right_x (array([  6,  23,  28,  40,  56,  59,  95, 104, 113]),)


hand_right_z (array([  8,  22,  24,  31,  36,  37,  65,  66,  78,  80,  90,  94, 102]),)


knee_left_x (array([  8,  24,  32,  37,  46,  49,  57,  61,  65,  81, 114]),)


wrist_left_x (array([  8,  20,  36,  43,  46,  80,  83,  91,  95, 103, 117, 118]),)


wrist_left_y (array([ 22, 101]),)


hip_left_x (array([  4,   6,  19,  31,  55,  73,  76,  86, 108, 114]),)


hand_left_x (array([ 45,  64,  86, 107]),)


head_y (array([ 20,  35,  44,  45,  47,  49,  50,  54,  57,  58,  63,  65,  68,
        71,  94, 114, 116]),)


knee_left_z (array([ 0,  7,  9, 28, 56, 96, 97]),)


shoulder_left_z (array([ 17,  28,  31,  32,  34,  38,  40,  44,  64,  69,  72,  74,  76,
        78,  93, 106, 117]),)


head_x (array([ 27,  46,  57,  59,  62,  68,  70,  87,  93,  95, 105]),)


foot_left_x (array([  2,  12,  37,  45,  63,  79,  84, 109, 113, 117]),)


foot_

## Voting & Evaluation ##

Now we evaluate the ensemble of classifiers on the same test set, allowing them to 'vote' on the correct label for a test_set gesture.

In [109]:
joints['ankle_left_x'].index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [111]:
# split recordNumbers into training & test sets
train, test = train_test_split(joints['ankle_left_x'].index.values, test_size = 0.2);

In [118]:
# Create dictionary of test gesture dataframes

In [137]:
# test on 'raw' - correct gestureLabel is 12
test_results = {}

# del classifiers['wrist_left_z']
for c in classifiers:
    sample = np.reshape(raw[c],[-1,1])
    print c
    test_results[c] = classifiers[c].predict(sample)
# raw['hip_center_x']

hand_right_y
hand_right_x
foot_left_x
hand_right_z
knee_left_x
wrist_left_x
wrist_left_y
knee_left_y
head_y
knee_left_z
shoulder_left_z
head_x
shoulder_left_x
shoulder_left_y
hand_left_y
foot_left_y
head_z
elbow_left_z
wrist_right_x
elbow_left_x
elbow_left_y
hip_center_x
hip_center_y
hip_center_z
shoulder_right_z
shoulder_right_x
shoulder_right_y
shoulder_center_x
shoulder_center_y
shoulder_center_z
wrist_right_z
elbow_right_x
wrist_right_y
elbow_right_z
foot_left_z
ankle_right_z
ankle_left_z
ankle_right_x
ankle_right_y
elbow_right_y
foot_right_x
ankle_left_x
ankle_left_y
hip_right_z
foot_right_y
foot_right_z
hip_left_z
hip_left_x
hip_left_y
spine_z
spine_y
spine_x
knee_right_z
knee_right_y
knee_right_x
hand_left_x
hip_right_x
hip_right_y


array(['11', '12', '8', '9'], 
      dtype='|S2')